In [1]:
import os
import datetime 
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

In [2]:
data_dir = 'C:\\Users\\cindy\\Desktop\\Mini-Project-1-Safety-Analysis-of-Autonomous-Vehicles\\simulation_results\\'

##### task 1.1

In [3]:
scenarios = ['clear-night', 'clear-noon', 'clear-sunset', 'haze-noon', 'haze-sunset', 'rain-noon']
csvs = ['ctl', 'cvip', 'traj']

In [ ]:
all_data = []
dfs = {}
for folder in os.listdir(data_dir):
    folder_path = os.path.join(data_dir, folder)
    if os.path.isdir(folder_path):
        for scenario in scenarios:
            if scenario in folder:
                folder = scenario  
                break  

        for file in os.listdir(folder_path):
            if file.endswith('.csv'):
                file_path = os.path.join(folder_path, file)
                df = pd.read_csv(file_path)
                for csv_type in csvs:
                    if csv_type in file:
                        file = csv_type
                        break 
                all_data.append((folder, file, df))  
                key = f"{folder}_{file}"  
                dfs[key] = df 

for i, (folder, file, df) in enumerate(all_data):
    print(f"Data from folder '{folder}', file '{file}':\n", df.head(), "\n")
    print("="*50)

In [ ]:
# List the first 5 rows of the ctl.csv, cvip.csv, and traj.csv with the clear-night weather condition.
scenario = 'clear-night'

for file_type in csvs:
    key = f"{scenario}_{file_type}" 
    df = dfs.get(key)  
    if df is not None:
        # print(df['agent_id'].nunique())
        print(f"Data from '{key}':\n", df.head())

##### task1.2

In [ ]:
# a. The duration of the scene
duration = 0
for scenario in scenarios:
    for file_type in csvs:
        key = f"{scenario}_{file_type}" 
        df = dfs.get(key)  
        if df is not None:
            start_time = df['ts'].min() 
            end_time = df['ts'].max()  
            duration = end_time - start_time

    print(f"Scene '{scenario}'")
    print(f"The duration of the scene: {duration} seconds")
    print("="*50)

In [ ]:
# b. Mean and standard deviation of the values of the features
features = ['throttle', 'steer', 'brake', 'cvip', 'x', 'y', 'v']
results_df = pd.DataFrame(index=features)

for scenario in scenarios:
    mean_std_values = {}
    for file_type in csvs:
        key = f"{scenario}_{file_type}" 
        df = dfs.get(key) 
        if df is not None:
            for feature in features:
                if feature in df.columns:
                    mean_value = df[feature].mean()
                    std_value = df[feature].std()
                    mean_std_values[feature] = f"{mean_value:.3f} ({std_value:.3f})"
    
    if mean_std_values:
        results_df[scenario] = pd.Series(mean_std_values)

results_df

##### task 1.3

In [ ]:
# colors, matching with scenarios
colors = ['tomato', 'crimson', 'darkorange', 'cornflowerblue', 'lightseagreen', 'lightslategrey']

for feature in features:
    plt.figure(figsize=(10, 6)) 
    plt.title(f"{feature.capitalize()} vs Time for different weather conditions")
    plt.xlabel("Time (ts)")
    plt.ylabel(f"{feature.capitalize()} value")

    for i, (scenario, color) in enumerate(zip(scenarios, colors)):
        df = None 
        for file_type in csvs:
            key = f"{scenario}_{file_type}"  
            df = dfs.get(key) 
            if df is not None and feature in df.columns:
                df['ts'] = pd.to_datetime(df['ts'], unit='s')
                plt.plot(df['ts'], df[feature], label=scenario, color=color, alpha=0.7)

    # second
    plt.gca().xaxis.set_major_locator(mdates.SecondLocator(interval=1000))  
    plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%H:%M:%S'))  

    plt.xticks(rotation=45)
    plt.legend(title="Weather Conditions", loc='lower right')
    plt.tight_layout()  
    plt.show()

##### task 1.4

##### Based on your intuition and life experience, which of the features do you think will change during an accident? How will the feature(s) change? 
1. `Throttle`: The throttle may decrease sharply just before the accident, as drivers usually slow down or stop when sensing a collision is imminent.
2. `Brake`: The brake input may increase dramatically during an accident, indicating the driver’s attempt to stop the vehicle.
3. `Steering`: The steering may show rapid changes during an accident, as the driver might try to swerve to avoid an object or another vehicle.
4. `Speed(v)`: Speed will typically decrease and may even reach zero as the vehicle comes to a stop due to the accident.
5. `Distance to other objects(cvip)`: The distance between the vehicle and other vehicles or objects will decrease rapidly as the accident approaches.
6. `x(horizontal direction)`: If the driver or the autonomous system attempts to swerve before the accident, x may undergo rapid changes.
7. `y(driving direction)`: If the vehicle decelerates or stops before the accident, y will gradually decrease and may eventually approach zero.

##### By looking at the plots you generated in Task 1.3, which weather condition(s) has an accident? 

##### task1.3 plot time as an ordinal variable

In [ ]:
offset = 0  

for feature in features:
    plt.figure(figsize=(10, 6)) 
    plt.title(f"{feature.capitalize()} vs Time for different weather conditions")
    plt.xlabel("Index")
    plt.ylabel(f"{feature.capitalize()} value")

    for i, (scenario, color) in enumerate(zip(scenarios, colors)):
        df = None 
        for file_type in csvs:
            key = f"{scenario}_{file_type}"  
            df = dfs.get(key) 
            if df is not None and feature in df.columns:
                plt.plot(df.index + offset, df[feature], label=scenario, color=color, alpha=0.7)
                offset += len(df)

    plt.xticks(rotation=45)  
    plt.legend(title="Weather Conditions", loc='lower right')
    plt.tight_layout()  
    plt.show()

##### task1.3 subplot

In [ ]:
ncols = 3  
nrows = (len(scenarios) + ncols - 1) // ncols  

for feature in features:
    fig, ax = plt.subplots(nrows, ncols, figsize=(12, 6 * nrows))  

    ax = ax.flatten()  
    for i, (scenario, color) in enumerate(zip(scenarios, colors)):
        df = None  
        for file_type in csvs:
            key = f"{scenario}_{file_type}"  
            df = dfs.get(key)  
            if df is not None and feature in df.columns:
                ax[i].plot(df['ts'], df[feature], label=scenario, color=color, alpha=0.7)
                ax[i].set_title(f"{feature.capitalize()} vs Time for {scenario}")
                ax[i].set_xlabel("Time (ts)")
                ax[i].set_ylabel(f"{feature.capitalize()} value")
                ax[i].legend(loc='lower right')

    for j in range(i + 1, len(ax)):
        fig.delaxes(ax[j])

    plt.tight_layout()  
    plt.show()